# 2D Stardist segmentation on 2D/3D/timelapse OMERO images

This notebook is used for Stardist segmentation. Some inspiration from the https://github.com/ome/omero-guide-cellprofiler/idr0002.ipynb

## TO DO
- Make a generic function for 2D segmentation for all slices independent of the shape of the image z,c,t
- Include a ID to all files uploaded to OMERO to make it more tracable
- Extend to handle multiple channels AND timepoints
- Check if we can overwrite label images if nesseary or ROIs


### Import Packages

In [1]:
# Import OMERO Python BlitzGateway
import omero
from omero.gateway import BlitzGateway
import ezomero
# Import Numpy
import numpy as np

# Import Python System Packages
import os
import tempfile
import pandas
import warnings

#stardist related
from stardist.models import StarDist2D
from csbdeep.utils import normalize
from stardist.plot import render_label
import matplotlib.pyplot as plt
from tifffile import imsave

#load stardist model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


### Set Temp Output Directory

In [2]:
import datetime

new_output_directory = os.path.normcase(tempfile.mkdtemp())
print(new_output_directory)
#create unique job id for reference based on date and time
job_id = str(datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
print(job_id)

c:\users\mwpaul\appdata\local\temp\tmpoq98sgaw
20250110165135


### Setup connection with OMERO

In [3]:
conn = BlitzGateway(host='localhost', username='root', passwd='omero', secure=True)
print(conn.connect())
conn.c.enableKeepAlive(60)

NameError: name 'BlitzGateway' is not defined

### Get info from the dataset

In [4]:
datatype = "dataset" # "plate", "dataset", "image"
data_id = 	502
nucl_channel = 0

#validate that data_id matches datatype
if datatype == "plate":
    plate = conn.getObject("Plate", data_id)
    print('Plate Name: ', plate.getName())
elif datatype == "dataset":
    dataset = conn.getObject("Dataset", data_id)
    print('Dataset Name: ', dataset.getName())
elif datatype == "image":
    image = conn.getObject("Image", data_id)
    print('Image Name: ', image.getName())

Dataset Name:  day7


### Run Stardist on the dataset

#### Function definitions

persistent_dict.py (63): Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.


#### code to run the analysis

In [6]:
import importlib
import src.ProcessImage as ProcessImage
importlib.reload(ProcessImage)
import pandas as pd

#get list of images to process
if datatype == "plate":
    wells = list(plate.listChildren())
    well_count = len(wells)
    images = []
    for count, well in enumerate(wells):
        print('Well: %s/%s' % (count + 1, well_count), 'row:', well.row, 'column:', well.column)
        # Load all images for a well if there are multiple
        fields = well.countWellSample()
        for field in range(fields):
            print('Field:', field)
            image = well.getImage(field)
            images.append(image)
elif datatype == "dataset":
    images = list(dataset.listChildren())
elif datatype == "image":
    images = [image]   

#process images
plate_statistics = []
for count,image in enumerate(images):
    print(f'Processing image {count+1}/{len(images)}: {image.getName()}')
    
    # Initialize processing
    img = ProcessImage.ProcessImage(conn, image, job_id, model)
    
    # Segment nuclei
    img.segment_nuclei(nucl_channel)
    
    # Save results
    img.save_segmentation_to_omero_as_attach(new_output_directory, desc)
    img.save_segmentation_to_omero_as_roi()
    #img.save_segmentation_to_omero_as_new_image(seg_img_name,desc)
    
    # Measure intensity
    img.measure_intensity()
    all_statistics_df = img.get_measurements_to_df()
    
    #save intensity measurements to OMERO
    all_statistics_df['imageID'] = image.getId()

    plate_statistics.append(all_statistics_df)
    image_id = image.getId()
    tabelid = ezomero.post_table(conn, object_type="Image", object_id=image.getId(), table = all_statistics_df,title=f"Nuclei_measurements_{job_id}_{image_id}")
    print('Created table ID:', tabelid)
    
# Concatenate all statistics into a single DataFrame
plate_statistics_df = pd.concat(plate_statistics, ignore_index=True)
tabelid = ezomero.post_table(conn, object_type="Dataset", object_id=data_id, table = plate_statistics_df, title=f"Nuclei_measurements_{job_id}_{data_id}")

     

File annotation ID: 1347
Created ROI with ID: 449


cast.py (1134): invalid value encountered in cast
cast.py (1153): invalid value encountered in cast


Created table ID: 1348
File annotation ID: 1349
Created ROI with ID: 450


cast.py (1134): invalid value encountered in cast
cast.py (1153): invalid value encountered in cast


Created table ID: 1350
File annotation ID: 1351
Created ROI with ID: 451


...

cast.py (1134): invalid value encountered in cast
cast.py (1153): invalid value encountered in cast


Created table ID: 1488
File annotation ID: 1489
Created ROI with ID: 520


cast.py (1134): invalid value encountered in cast
cast.py (1153): invalid value encountered in cast


Created table ID: 1490


cast.py (1134): invalid value encountered in cast
cast.py (1153): invalid value encountered in cast


### Delete attachements from project

In [49]:
datatype = "dataset" # "plate", "dataset", "image"
data_id = 	502

def ensure_list(obj):
    if not obj:
        return []
    if isinstance(obj, list):
        return obj
    return [obj]

if datatype == "dataset":
    images = list(dataset.listChildren())
    image_count = len(images)
    plate_statistics = []
    to_delete = []
    for count in range(image_count):
        image = images[count]
        i = conn.getObject("Image", image.getId())
        print('Image Name:', i.getName())
        
        for ann in i.listAnnotations():
            link_id = ann.link.id  # sometimes single, sometimes list
            link_ids = ensure_list(link_id)

            for lid in link_ids:
                to_delete.append(lid)
    conn.deleteObjects("ImageAnnotationLink", to_delete, wait=True)

Image Name: 14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 [14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 (series 01)].tif
Image Name: 14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 [14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 (series 02)].tif
Image Name: 14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 [14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 (series 03)].tif
Image Name: 14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 [14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 (series 04)].tif
Image Name: 14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 [14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 (series 05)].tif
Image Name: 14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 [14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 (series 06)].tif
Image Name: 14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 [14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 (series 07)].tif
Image Name: 14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 [14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 (series 08)].tif
Image Name: 14-6-24_FUCCI_PDLO_Day7_plat

### Delete ROIs from project

In [50]:
datatype = "dataset" # "plate", "dataset", "image"
data_id = 	502

if datatype == "dataset":
    images = list(dataset.listChildren())
    image_count = len(images)
    plate_statistics = []
    for count in range(image_count):
        image = images[count]
        roi_service = conn.getRoiService()
        result = roi_service.findByImage(image.getId(), None)
        roi_ids = [roi.id.val for roi in result.rois]
        conn.deleteObjects("Roi", roi_ids)
